In [ ]:
"""
Input: string, prompt from user.

Output: string, tags from four classes.

"""

In [12]:
"""
LLM usage
"""


from llama_cpp import Llama, LlamaGrammar
from typing import Iterable, List


In [113]:
labels = {
    "Process structure",
    "Distribution of cases over paths",
    "Throughput time of cases",
    "Resource utilization rate",
    "other",
}
prompt_PATH = "./systemprompt.txt"



In [114]:
"""
llm = Llama.from_pretrained(
	    repo_id="unsloth/Qwen3-4B-GGUF",
	    filename="Qwen3-4B-Q5_K_M.gguf",
    )

"""

llm_instruct = Llama.from_pretrained(
    repo_id="matrixportalx/Qwen2.5-7B-Instruct-GGUF",
	filename="qwen2.5-7b-instruct-q4_k_m.gguf",
)

llama_model_load_from_file_impl: using device Metal (Apple M3 Pro) - 28747 MiB free
llama_model_loader: loaded meta data with 34 key-value pairs and 339 tensors from /Users/silei/.cache/huggingface/hub/models--matrixportalx--Qwen2.5-7B-Instruct-GGUF/snapshots/bc4565de1b0c43d1b897a5cac2c510979e4da5bb/./qwen2.5-7b-instruct-q4_k_m.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Qwen2.5 7B Instruct
llama_model_loader: - kv   3:                           general.finetune str              = Instruct
llama_model_loader: - kv   4:                           general.basename str              = Qwen2.5
llama_model_loader: - kv   5:  

In [74]:


def make_enum_grammar(labels: Iterable[str], add_other: bool = True) -> LlamaGrammar:
    """
    Create a llama.cpp GBNF grammar that forces output to be exactly one of the given labels.

    Example output:
        root ::= label
        label ::= "a" | "b" | "c" | "other"
    """
    # normalize, deduplicate, keep order
    seen = set()
    items: List[str] = []
    for x in labels:
        x = str(x).strip()
        if not x:
            continue
        if x not in seen:
            seen.add(x)
            items.append(x)

    if add_other and "other" not in seen:
        items.append("other")

    if not items:
        raise ValueError("labels is empty after normalization")

    def gbnf_quote(s: str) -> str:
        # Escape for GBNF string literal
        # (quotes and backslashes are the main ones you may hit)
        s = s.replace("\\", "\\\\").replace('"', '\\"')
        return f"\"{s}\""

    options = " | ".join(gbnf_quote(x) for x in items)

    grammar_str = f"""
root ::= label
label ::= {options}
""".strip()
    
    grammar = LlamaGrammar.from_string(grammar_str)

    return grammar


In [ ]:
def semantic_recognition(
        prompt_from_user: str,
        llm: Llama
) -> str:
    semantic_tag = ""
    messages = []
    grammar = make_enum_grammar(labels, True)
    

    with open(prompt_PATH, "r", encoding="utf-8") as f:
        system_prompt = f.read()

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt_from_user},
    ]
    
    res = llm.create_chat_completion(
	    messages,
        max_tokens=512,
        temperature=0.1,
        repeat_penalty=1.00,
        grammar=grammar,
        )

    """"
    TODO: add memory storage in this part + monitoring maybe
    """
    
    semantic_tag = res["choices"][0]["message"]["content"]

    return semantic_tag 

In [102]:
User_prompt_d = "Please generate 100 synthetic event logs for a process mining course. The goal is to help students analyze different process variants and how many cases follow each variant. Make sure the event logs contain at least 4 distinct paths with clearly different frequencies (e.g., one dominant path and several rare paths)."

User_prompt_p = "Could you please help me to create 100 eventlogs which will be used to let student know how to explore the structure of process from event logs."

User_prompt_r = "Could you create 100 event logs for teaching process mining? I want students to explore resource behavior from the logs, such as workload distribution, busy vs idle periods, and whether certain resources are over-utilized. Please include multiple resources and realistic handovers between them."

User_prompt_o = "Hello"

In [ ]:
resu= semantic_recognition(User_prompt_d, llm_instruct)
print(resu)

Llama.generate: 342 prefix-match hit, remaining 66 prompt tokens to eval
llama_perf_context_print:        load time =    5195.82 ms
llama_perf_context_print: prompt eval time =    2230.64 ms /    66 tokens (   33.80 ms per token,    29.59 tokens per second)
llama_perf_context_print:        eval time =     222.19 ms /     5 runs   (   44.44 ms per token,    22.50 tokens per second)
llama_perf_context_print:       total time =    2513.21 ms /    71 tokens
llama_perf_context_print:    graphs reused =          4


Distribution of cases over paths


In [ ]:
from __future__ import annotations

from dataclasses import dataclass
from typing import Any, Dict, List, Optional, Union
import json


@dataclass
class ParsedChatCompletion:
    content: str
    finish_reason: Optional[str]
    usage: Dict[str, int]
    tool_calls: List[Dict[str, Any]]
    raw_message: Dict[str, Any]
    choice_index: int = 0


def parse_chat_completion(
    res: Dict[str, Any],
    choice_index: int = 0,
    *,
    strict: bool = False,
) -> ParsedChatCompletion:
    """
    Parse a llama.cpp / llama-cpp-python create_chat_completion() response.

    Extracts:
      - content (assistant text)
      - tool_calls (if any)
      - usage (prompt_tokens/completion_tokens/total_tokens)
      - finish_reason

    Args:
        res: Response dict returned by create_chat_completion()
        choice_index: Which choice to parse (default 0)
        strict: If True, raise on missing fields; else be tolerant.

    Returns:
        ParsedChatCompletion
    """
    if not isinstance(res, dict):
        raise TypeError(f"res must be dict, got {type(res)}")

    choices = res.get("choices")
    if not isinstance(choices, list) or len(choices) == 0:
        if strict:
            raise ValueError("Response has no choices")
        choices = []

    if choice_index >= len(choices):
        if strict:
            raise IndexError(f"choice_index={choice_index} out of range for choices (len={len(choices)})")
        # fabricate an empty choice
        choice = {}
    else:
        choice = choices[choice_index] or {}

    finish_reason = choice.get("finish_reason")

    # message payload (chat.completion)
    message = choice.get("message") or {}
    if not isinstance(message, dict):
        message = {}

    # content: some tool-calling responses may have empty content
    content = message.get("content")
    if content is None:
        content = ""
    if not isinstance(content, str):
        # sometimes content can be non-string; coerce safely
        content = str(content)

    # tool_calls: OpenAI-style tools
    tool_calls = message.get("tool_calls") or []
    if tool_calls is None:
        tool_calls = []
    if not isinstance(tool_calls, list):
        tool_calls = [tool_calls]

    # Back-compat: some implementations may put "function_call" instead of tool_calls
    # We'll normalize into tool_calls-like structure
    function_call = message.get("function_call")
    if function_call and not tool_calls:
        # expected shape: {"name": "...", "arguments": "..."}
        if isinstance(function_call, dict) and "name" in function_call:
            tool_calls = [{
                "type": "function",
                "function": {
                    "name": function_call.get("name"),
                    "arguments": function_call.get("arguments", ""),
                },
            }]

    # usage
    usage = res.get("usage") or {}
    if not isinstance(usage, dict):
        usage = {}

    def _int_or_zero(x: Any) -> int:
        try:
            return int(x)
        except Exception:
            return 0

    usage_norm = {
        "prompt_tokens": _int_or_zero(usage.get("prompt_tokens", 0)),
        "completion_tokens": _int_or_zero(usage.get("completion_tokens", 0)),
        "total_tokens": _int_or_zero(usage.get("total_tokens", 0)),
    }

    if strict:
        # basic strict validation
        if "choices" not in res:
            raise ValueError("Missing 'choices'")
        if "usage" not in res:
            raise ValueError("Missing 'usage'")
        if not isinstance(choice.get("message"), dict):
            raise ValueError("Missing/invalid 'choice.message'")

    return ParsedChatCompletion(
        content=content,
        finish_reason=finish_reason,
        usage=usage_norm,
        tool_calls=tool_calls,
        raw_message=message,
        choice_index=choice_index,
    )


def extract_tool_call_args(tool_call: Dict[str, Any]) -> Dict[str, Any]:
    """
    Helper: parse tool_call.function.arguments into dict if it's a JSON string.
    Returns {} if parse fails.
    """
    fn = tool_call.get("function") or {}
    args = fn.get("arguments", "")
    if isinstance(args, dict):
        return args
    if not isinstance(args, str):
        return {}
    args = args.strip()
    if not args:
        return {}
    try:
        return json.loads(args)
    except Exception:
        return {}


# --- Example usage ---
# parsed = parse_chat_completion(res)
# print(parsed.content, parsed.finish_reason, parsed.usage)
# if parsed.tool_calls:
#     for tc in parsed.tool_calls:
#         name = (tc.get("function") or {}).get("name")
#         args = extract_tool_call_args(tc)
#         print("TOOL:", name, args)
